Example code from https://www.hackster.io/news/easy-tinyml-on-esp32-and-arduino-a9dbc509f26c

Guide for tinymlgen at https://github.com/eloquentarduino/tinymlgen

In [1]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tinymlgen import port

def get_model():
    SAMPLES = 1000
    np.random.seed(1337)
    x_values = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)
    # shuffle and add noise
    np.random.shuffle(x_values)
    y_values = np.sin(x_values)
    y_values += 0.1 * np.random.randn(*y_values.shape)

    # split into train, validation, test
    TRAIN_SPLIT =  int(0.6 * SAMPLES)
    TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)
    x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
    y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

    # create a NN with 2 layers of 16 neurons
    model = tf.keras.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    model.fit(x_train, y_train, epochs=200, batch_size=16,
                        validation_data=(x_validate, y_validate))
    return model


# model = get_model()
# c_code = port(model, pretty_print=True)
# print(c_code)

model = get_model()
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("sine_model_quantized.tflite", "wb").write(tflite_model)

Epoch 1/200
38/38 [==============================] - 3s 26ms/step - loss: 0.4270 - mae: 0.5699 - val_loss: 0.4212 - val_mae: 0.5633
Epoch 2/200
38/38 [==============================] - 0s 8ms/step - loss: 0.3310 - mae: 0.4994 - val_loss: 0.3534 - val_mae: 0.4993
Epoch 3/200
38/38 [==============================] - 0s 9ms/step - loss: 0.2726 - mae: 0.4556 - val_loss: 0.2899 - val_mae: 0.4545
Epoch 4/200
38/38 [==============================] - 0s 8ms/step - loss: 0.2262 - mae: 0.4123 - val_loss: 0.2384 - val_mae: 0.4309
Epoch 5/200
38/38 [==============================] - 0s 9ms/step - loss: 0.1968 - mae: 0.3855 - val_loss: 0.2037 - val_mae: 0.3887
Epoch 6/200
38/38 [==============================] - 0s 7ms/step - loss: 0.1789 - mae: 0.3656 - val_loss: 0.1801 - val_mae: 0.3663
Epoch 7/200
38/38 [==============================] - 0s 8ms/step - loss: 0.1678 - mae: 0.3497 - val_loss: 0.1755 - val_mae: 0.3623
Epoch 8/200
38/38 [==============================] - 0s 9ms/step - loss: 0.1581 - 

INFO:tensorflow:Assets written to: C:\Users\jared\AppData\Local\Temp\tmpi8jsl9x7\assets


INFO:tensorflow:Assets written to: C:\Users\jared\AppData\Local\Temp\tmpi8jsl9x7\assets


3168

Converting of model to C array (Run below line on bash)

xxd -i sine_model_quantized.tflite > sine_model_quantized.cc